## Carga de datos

In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
import plotly.express as px


In [ ]:
UNINTRESTING_OMEGAS = (0.1, 0.2, 0.3, 0.4, 0.6, 0.65, 0.8, 0.9)
INTRESTING_OMEGAS = (0.5, 0.55)


## CARGA DE DATOS PARA MEDIR DIFERENCIAS ENRTRE CRITERIOS
exports_diff_dfs = []
exports_global_dfs = []
exports_local_dfs = []
for omega in list(UNINTRESTING_OMEGAS) + list(INTRESTING_OMEGAS):
    exports_global_df = pd.read_csv(f"experimentacion/metrics-original-data-true-{omega}/exports.csv")
    exports_global_df["omega"] = omega
    exports_global_df.set_index(["country", "generation"], inplace=True)
    exports_global_dfs.append(exports_global_df)

    exports_local_df = pd.read_csv(f"experimentacion/metrics-original-data-false-{omega}/exports.csv")
    exports_local_df["omega"] = omega
    exports_local_df.set_index(["country", "generation"], inplace=True)
    exports_local_dfs.append(exports_local_df)

    exports_diff_df = exports_global_df - exports_local_df
    exports_diff_df["omega"] = omega

    exports_diff_dfs.append(exports_diff_df)

exports_diff_df = pd.concat(exports_diff_dfs)
exports_diff_df.reset_index(inplace=True)

exports_global_df = pd.concat(exports_global_dfs)
exports_global_df.reset_index(inplace=True)

exports_local_df = pd.concat(exports_local_dfs)
exports_local_df.reset_index(inplace=True)

exports_diff_df

In [ ]:
exports_global_df_copy = exports_global_df.copy()
exports_global_df_copy["dynamic"] = True
exports_local_df_copy = exports_local_df.copy()
exports_local_df_copy["dynamic"] = False
exports_df = pd.concat([exports_local_df_copy, exports_global_df_copy])
exports_df

In [ ]:
original_data = pkl.load(open("data/original_data.pkl", "rb"))
len(original_data["products"]), len(original_data["countries"])

In [ ]:
print(original_data["countries"])

## Gráficos PS fijo

In [ ]:
paises = set(["Argentina", "Brazil", "Korea Rep.", "Germany"])

# Criterio local
df = exports_local_df[
    exports_local_df.country.isin(paises) &
    exports_local_df.omega.isin([0.4,0.5,0.55,0.6,0.65]) &
    (exports_local_df.generation <= 10)
]
df

In [ ]:
fig = px.line(df,
            x="generation",
            y="count_exports",
            color="omega",
            facet_col="country",
            facet_col_wrap=2,
            height=600,
#            title="Cantidad de exportaciones en función de la generación por país y omega para variante local",
            labels={
                "count_exports": "Exports quantity",
                "generation": "Simulation steps",
                "country": "Country",
                "omega": "Threshold",
            })
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
#fig.add_vline(x=10, annotation_text="Fin sugerido de la simulación", annotation_position="bottom right")
fig.add_hline(y=len(original_data["products"]), annotation_text="Products quantity", opacity=.25)
fig.show()

In [ ]:
from metrics.product_space_graph import ProductSpaceGraph
graph_local = ProductSpaceGraph("experimentacion/original-data-local-0.55/exports_history.pkl")

In [ ]:
graph_local.plot("Argentina", legend_title="Simulation steps", figsize=(9,7))

## Gráficos PS fijo vs dinámico

In [ ]:
df_ = (exports_df
 .set_index(["generation", "country", "omega"])
 .loc[max(exports_diff_df.generation)]
 .reset_index()
 .sort_values("omega")
)

mean_df = df_.groupby(["omega","dynamic"]).mean().reset_index()
mean_df["country"] = "World mean"
df_ = pd.concat([df_, mean_df])
df_ = df_[df_.country.isin(["Argentina", "World mean", "Germany"])]
fig = px.line(
    df_,
    x="omega",
    y="count_exports",
    color="country",
    line_dash="dynamic",
    markers=True,
    height=600,
    range_y=[0,800],
#    title="Exportaciones por país y variante de simulación",
    labels={
        "count_exports": "Exports quantity",
        "country": "Country",
        "omega": "Threshold",
        "dynamic": "Dynamic?"
    },
)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
fig.add_hline(y=len(original_data["products"]), annotation_text="Products quantity", opacity=.25)
fig.show()

In [ ]:
max_diff_df = (
    exports_diff_df
    .set_index(["generation", "country", "omega"])
    .loc[max(exports_diff_df.generation)]
    .max(axis=1)
)
max_diff_df.rename("difference", inplace=True)
fig = px.box(
    max_diff_df.reset_index(),
    x="omega",
    y="difference",
#    title=f"Distribucion de la ventaja de la variante global respecto a la variante local<br>según omega para todos los países (ultima generacion)",
    labels={
        "difference": "Difference between models",
        "omega": "Threshold"})
fig.show()